<h1> Hyperledger Sawtooth </h1>

* Exemplos de implementações:
<ol>
<li><a href="https://github.com/yellowssi/SeaStorage">Implementação decentralizada para file storage</a></li>
<li><a href="https://github.com/Kerala-Blockchain-Academy/hygieia">Implementação de saúde (doação de orgãos)</a></li>
<li><a href="https://github.com/hyperledger-archives/sawtooth-marketplace"> Implementação marketplace</li>

<li><a href="https://github.com/hyperledger-archives/sawtooth-supply-chain">Supply chain</li>

<li><a href="https://github.com/hyperledger-archives/sawtooth-private-utxo">Sei la o que é isso</li>

<li></li>
</ol>


<a href="https://events19.linuxfoundation.org/wp-content/uploads/2017/12/Blockchain-Technology-Partners-OSS-EU-2018-Session_Duncan-Johnston-Watt.pdf
">Documento pode ajudar a entender melhor como o VALIDADOR  funciona</a>

Aparentemente é possível utilizar um validador para diversas aplicações (TransactionProcessors)-> Definindo no validador uma transaction family para cada aplicação.

--> Discussão sobre o design de dados escolhido no sawtooth: https://www.lfdecentralizedtrust.org/blog/2018/01/16/data-model-considerations-in-hyperledger-sawtooth


* Aparentemente o exemplo da simple-wallet parece ser útil e simples o suficiente para explorar: https://github.com/askmish/sawtooth-simplewallet

<h1> Hyperledger fabric </h1>

* Parece simples, mais utilizado e tem mais tutorials

* Dois algoritmos de consenso: pBFT e RAFT

* Pelo que entendi, o consenso ocorre no orderer. ?? Porém o consenso é apenas sobre a ordem em que as transações são armazenadas em um bloco ?? No entanto, garantir a ordem já é suficiente para nosso modelo ! pois queremos que todos os nós envolvidos informem seu QoS de forma honesta.

* problema: nós precisam se conhecer antes de formar a blockchain...

* Limitation of Hyperledger Fabric

    1 Hyperledger Fabric is a robust and flexible platform for developing blockchain applications, but like any technology, it has certain limitations:

    2 Scalability: Hyperledger Fabric is designed for permissioned networks, where the participants are known and trusted, which can limit its scalability for large-scale public networks.
    
    3 Performance: The performance of Hyperledger Fabric can be impacted by factors such as network size, network configuration, and the complexity of chaincode, which can limit its ability to handle high volumes of transactions.

    4 Complexity: Setting up and configuring a Hyperledger Fabric network can be complex, requiring a deep understanding of the technology and its components.
    
    5 Compatibility: Hyperledger Fabric is designed to be used with specific programming languages, such as Go and JavaScript, which can limit its compatibility with other technologies and programming languages.
    
    6 Cost: Running a Hyperledger Fabric network requires infrastructure and resources, which can add costs to the deployment and operation of blockchain applications.

    7 Interoperability: Hyperledger Fabric is designed to be used within a single network, and its interoperability with other blockchain platforms is limited.

<h1> O que eu quero </h1>

* Uma blockchain onde cada ledger tenha um conjunto de nós validadores, que podem mudar com o tempo.

* Ou, uma blockchain onde cada blockchain tenha um conjunto de nós validadores que podem mudar com o tempo.

* O que percebi, redes permissionadas precisa-se conhecer os nós de antemão e as não permissionadas aparentemente são mais livres dessa limitação.

* Get dockerfile of an image: sudo docker history --no-trunc 7d249ca8774e >> sawtooth_validator_dockerfile.dockerfile


<h2> NEXT </h2>

* Combinar os diversos containers em um apenas (validador, settings, rest, consensus) --> diminuir exposição de portas e permitir mais blcockchains ++ diminuir recursos por blockchain.

<H3> Comandos docker CLI <H3>

- Ver todos os containers (parados tbm):

```
sudo docker ps -a
```

- Criar imagem a partir de dockerfile:

```
sudo docker build --debug --tag 'helloworldv1' .
```